In [23]:
sdata.tables["proteomics"]

AnnData object with n_obs × n_vars = 18 × 4531
    obs: 'filename', 'sample_name', '384wp_row', '384wp_col', '384_wp_well', 'slide_number', 'RCN', 'replicate', 'Biopsy_type', 'CellularNeighborhood', 'shape_key', 'region'
    var: 'Protein.Ids', 'Protein.Names', 'First.Protein.Description', 'Protein.Group'
    uns: 'spatialdata_attrs'

In [2]:
#imports
import spatialdata as sd
import pandas as pd
import anndata as ad
from loguru import logger
import time
import napari_spatialdata

# Load dataset

In [3]:
#paths
sdata_big = "../data/sdata/20241201_1730_sdata_fullres.zarr"
path_to_imaging_table = "/Users/jnimoca/Jose_BI/4_Projects/P26_BCCN/P26E10_ImageAnalysis/data/processed/20241002_991.h5ad"
path_to_raw_imaging_data = "/Volumes/Extreme SSD/P26_BCCN/P26E07_E08_E09_SOPA_Gates/quantification/991.csv"

In [4]:
# sdata = sd.read_zarr(sdata_big)
sdata = sd.SpatialData.read(sdata_big)

# imaging = pd.read_csv(path_to_raw_imaging_data)

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [6]:
imaging.head()

In [5]:
def read_quant(csv_data_path) -> ad.AnnData:
    """
    Read the quantification data from a csv file and return an anndata object
    :param csv_data_path: path to the csv file
    :return: an anndata object
    """
    logger.info(" ---- read_quant : version number 1.1.0 ----")
    time_start = time.time()

    assert csv_data_path.endswith('.csv'), "The file should be a csv file"
    df = pd.read_csv(csv_data_path)
    df.index = df.index.astype(str)

    meta_columns = ['CellID', 'Y_centroid', 'X_centroid',
        'Area', 'MajorAxisLength', 'MinorAxisLength', 'Eccentricity',
        'Orientation', 'Extent', 'Solidity']
    assert all([column in df.columns for column in meta_columns]), "The metadata columns are not present in the csv file"

    metadata = df[meta_columns]
    data = df.drop(columns=meta_columns)
    variables = pd.DataFrame(
        index=data.columns,
        data={"math": [column_name.split("_")[0] for column_name in data.columns],
            "marker": ["_".join(column_name.split("_")[1:]) for column_name in data.columns]})

    adata = ad.AnnData(X=data.values, obs=metadata, var=variables)
    logger.info(f" {adata.shape[0]} cells and {adata.shape[1]} variables")
    logger.info(f" ---- read_quant is done, took {int(time.time() - time_start)}s  ----")
    return adata

In [6]:
adata = read_quant(path_to_raw_imaging_data)

2024-12-02 16:00:01.060 | INFO     | __main__:read_quant:7 -  ---- read_quant : version number 1.1.0 ----
2024-12-02 16:00:05.675 | INFO     | __main__:read_quant:27 -  685805 cells and 75 variables
2024-12-02 16:00:05.675 | INFO     | __main__:read_quant:28 -  ---- read_quant is done, took 4s  ----


In [ ]:
def filter_by_adataobs ( adata, key="Area", label="Area_boolean_250") -> ad.AnnData : 

In [ ]:
def check_filter (sdata, column_in_adata_to_check):
    # plot points from column in napari interactive
    # use shapes (from segmentation)
    # use points (sdata.points.element)

In [7]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/1_Pipelines/openDVP/data/sdata/20241201_1730_sdata_fullres.zarr
├── Images
│     └── 'image': DataArray[cyx] (15, 44470, 73167)
├── Labels
│     └── 'mask': DataArray[yx] (44470, 73167)
├── Shapes
│     └── 'Primary_contours': GeoDataFrame shape: (347, 6) (2D shapes)
└── Tables
      └── 'proteomics': AnnData (18, 4531)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), mask (Labels), Primary_contours (Shapes)

sdata['segmentation_mask'] = sd.to_polygons(sdata['mask'])

In [9]:
adata.obs["labels"] = "mask"
adata.obs["labels"] = adata.obs["labels"].astype("category")
table = sd.models.TableModel.parse(adata, region = "mask", region_key = "labels", instance_key = "CellID")
sdata["imaging"] = table

In [10]:
sdata

SpatialData object, with associated Zarr store: /Users/jnimoca/Jose_BI/1_Pipelines/openDVP/data/sdata/20241201_1730_sdata_fullres.zarr
├── Images
│     └── 'image': DataArray[cyx] (15, 44470, 73167)
├── Labels
│     └── 'mask': DataArray[yx] (44470, 73167)
├── Shapes
│     └── 'Primary_contours': GeoDataFrame shape: (347, 6) (2D shapes)
└── Tables
      ├── 'imaging': AnnData (685805, 75)
      └── 'proteomics': AnnData (18, 4531)
with coordinate systems:
    ▸ 'global', with elements:
        image (Images), mask (Labels), Primary_contours (Shapes)
with the following elements not in the Zarr store:
    ▸ imaging (Tables)

In [ ]:
import napari_spatialdata
interactive = napari_spatialdata.Interactive(sdata)
interactive.run()

/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/napari/_vispy/layers/scalar_field.py:199: UserWarning: data shape (44470, 73167) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
2024-12-02 16:02:22.683 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-12-02 16:02:22.683 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
202

In [ ]:
print(sd.__version__)

0.2.5.post0


/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/napari/_vispy/layers/scalar_field.py:199: UserWarning: data shape (44470, 73167) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
2024-12-02 15:48:58.190 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
2024-12-02 15:48:58.192 | INFO     | napari_spatialdata._view:_on_layer_update:355 - Updating layer.
/opt/homebrew/Caskroom/mambaforge/base/envs/spatialdata/lib/python3.10/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


: 